# Mammalia-Macaque-Dominance Network

Laura Silvana Alvarez Luque - Daniel Losada Molina


The dominance relations between *Macaca fuscata* females were determined based on approach-retreat episodes around the food. The dominance range order was arranged based on these dyadic relations. Dominance is defined as "Physical contact unique to aggressive dominance interactions such as biting, head butting, fighting"

- **Vertices:** 62 Macaques.
- **Edges:** 1.167 edges representing the pairwise dominance between macaques.
- **Weights:** Frequency of the dominance.
- **Notes:**  This is a animal interation network, undirected and weighted.

![Macaca Fuscata](./imgs/Macaca.png)


In [1]:
import zipfile

import matplotlib.pyplot as plt
import igraph as ig
import plotly.graph_objects as go
import networkx as nx

# Functions

In [ ]:
def unzip_file(zip_path, extract_to):
    """
    Unzip a file from zip_path into the extract_to directory.
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def plot_simple_graph(g):
    # Layout for node positions
    layout = g.layout("fr")  # Fruchterman-Reingold (force-directed)

    # Plot using igraph's built-in plot function
    ig.plot(
        g,
        layout=layout,
        vertex_label=g.vs.indices,  # show node indices
        edge_width=[w for w in g.es["weight"]],
        bbox=(800, 800),
        margin=50
    )

def plotly_graph(g):    
    # Convert to NetworkX
    G_nx = g.to_networkx()

    # Get layout positions
    pos = nx.spring_layout(G_nx)

    # Prepare edge lines
    edge_x = []
    edge_y = []
    for u, v in G_nx.edges():
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines',
        showlegend=False
    )

    # Create "hover points" at edge midpoints
    edge_hover_x = []
    edge_hover_y = []
    edge_text = []

    for u, v, data in G_nx.edges(data=True):
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_hover_x.append((x0 + x1) / 2)
        edge_hover_y.append((y0 + y1) / 2)
        weight = data.get("weight", 1)
        edge_text.append(f"{u} — {v}<br>weight: {weight}")

    edge_hover_trace = go.Scatter(
        x=edge_hover_x,
        y=edge_hover_y,
        mode='markers',
        hoverinfo='text',
        text=edge_text,
        marker=dict(size=5, color='rgba(0,0,0,0)'),  # invisible
        showlegend=False
    )

    # Prepare node positions
    node_x = []
    node_y = []
    node_text = []
    for node in G_nx.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(f"Node {node}")

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        text=[str(n) for n in G_nx.nodes()],
        textposition='top center',
        hoverinfo='text',
        marker=dict(
            size=10,
            color='lightblue',
            line_width=1
        )
    )

    # Combine everything
    fig = go.Figure(
        data=[edge_trace, edge_hover_trace, node_trace],
        layout=go.Layout(
            title='Dominance Network of Macaques',
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=5, r=5, t=40),
            xaxis=dict(showgrid=False, zeroline=False),
            yaxis=dict(showgrid=False, zeroline=False)
        )
    )

    fig.show()

# 1. Input data

In [ ]:
unzip_file('./mammalia-macaque-dominance.zip', './data')

In [ ]:
#%%capture
g = ig.Graph.Read_Ncol("data/mammalia-macaque-dominance.edges", weights=True, directed=False)
print(g.summary())


In [ ]:
#plotly_graph(g)

![Macaca Fuscata](./imgs/Graph.png)


We are dealing with a highly connected graph, as we can see by the edge-vertex ratio:

$ \frac{|E|}{|V|} = \frac{1167}{62} \approx 18.82 $

This means that in average, we have 18.82 edges per vertex.

In [14]:
components_weak = g.components()
print("Weakly connected components: ", len(components_weak))

components_strong = g.components(mode="strong")
print("Strongly connected components: ", (components_strong))

density = g.density(loops=True)
print(density)

Weakly connected components:  1
Strongly connected components:  Clustering with 62 elements and 1 clusters
[0] 1, 2, 3, 8, 10, 11, 12, 13, 14, 16, 18, 20, 21, 22, 23, 24, 26, 30, 31,
    33, 35, 36, 37, 38, 40, 42, 43, 45, 47, 48, 49, 50, 51, 52, 55, 56, 57,
    58, 59, 29, 41, 44, 53, 61, 5, 6, 17, 19, 25, 27, 28, 32, 34, 39, 46, 54,
    60, 62, 4, 9, 15, 7
0.5975422427035331
